# Downsampling
Downsampling reduces point cloud. The reason for using downsampling are as follows:

- Saving memory and processing time : When processing a point cloud with many points, a huge amount of memory and calculation time may be required during processing. By performing down-sampling for such point clouds, we can reduce the burden during processing.
- Reducing the complexity of point clouds: Some points in a point cloud are redundant for processing. Downsampling can reduce such points.

This section introduce the following downsampling methods. 
- Random Sampling
- FPS (Furthest point sampling)
- Voxel grid sampling

In [1]:
%load_ext autoreload
%autoreload 2

## Random sampling
Random sampling sample points without

## FPS (Furthest point sampling)
FPSは反復的にサンプリングを行い、反復時にサンプリングされた点の中から最も遠い点を新たなサンプル点として保存する手法である。この手法では、決められた点の数になるまで点をサンプリングするため、サンプリング点の数を指定しながらも均一な点群を取得したい場合に有用である。

本subsectionで使用するコードは以下の通り。

In [4]:
# for FPS
from tutlibs.sampling import furthest_point_sampling
from tutlibs.operator import gather


# for description
import numpy as np
from tutlibs.io import Points as io
from tutlibs.visualization import JupyterVisualizer as jv
from tutlibs.transformation import Transformation as tr
import inspect

In [5]:
# load point cloud data.
xyz, _, _ = io.read('../data/bunny_pc.ply')

# get sample indices from FPS function
idxs = furthest_point_sampling(xyz, 500)

# get sample points
fps_xyz = gather(xyz, idxs)

# visualize samples and origin.
fps_xyz = tr.translation(fps_xyz, np.array([1, 0, 0]))
obj_points = jv.point(xyz)
obj_fps_points = jv.point(fps_xyz)
jv.display([obj_points, obj_fps_points])

Output()

上記のコードの出力では、x軸方向に0~1の間にある点群がFPS前の点群、1~2の間にある点群がFPS後の点群を示す。点は指定された数(500個)までダウンサンプリングされている。FPSを実装したfurthest_point_sampling関数では、点群座標値`xyz`とダウンサンプリング数`500`からFPSを介して、`xyz`中のサンプルに対応するインデックス値`idxs`を取得する。

次に、アルゴリズムを確認するため、FPS関数の中身を以下に示す。

In [6]:
print(inspect.getsource(furthest_point_sampling))

def furthest_point_sampling(coords: np.ndarray, num_sample: int) -> np.ndarray:
    """Furthest point sampling

    Args:
        coords: coords (N, C)
        num_sample: number of sammple

    Returns:
        indices: sample indices (num_sample)
    """
    N, _ = coords.shape

    min_square_dists = np.full(N, 2**16-1, dtype=np.float32)
    sample_indices = np.zeros(num_sample, dtype=np.int32)

    # Get first index
    sample_indices[0] = 0
    for i in range(1, num_sample):
        # compute square distances between coords and previous sample.
        previous_sample = coords[sample_indices[i-1]]
        relative_coords = coords - previous_sample[np.newaxis, :]
        square_dists = np.sum(relative_coords**2, axis=1)

        # update minimum distance between coords and samples.
        min_dist_mask = square_dists < min_square_dists
        min_square_dists[min_dist_mask] = square_dists[min_dist_mask]

        # get new furthest point (sample) index.
        sample_indices[i] =

上記実装では、返り値がFPSのサンプル(`coords`のインデックス配列)となる。
アルゴリズムとしては、指定された点の数だけ点をサンプリングするまで反復処理を行う。反復処理は以下の通り。

1. 現在の反復までで得た全てのサンプリング点のいづれからも最遠である点を探す。探す点は、まだサンプリング点として割り当てられていない点である必要がある。
   1. 点群の点ごとに最も近いサンプル点との距離を保存する距離配列`min_square_dists`の中で最も距離のある点を探す。
2. 最遠である点を新たなサンプリング点として追加する。
   1. 追加先は`sample_indices`


## Voxel grid sampling
Voxel grid samplingは点群からボクセルグリッドに従った点をサンプリングするための手法である。この手法では、ボクセルグリッドに沿った点をサンプリングするため、点間距離が等間隔に近い点群を取得することができる。

In [7]:
# for voxel grid sampling
import numpy as np
from tutlibs.sampling import voxel_grid_sampling

# for description
from tutlibs.io import Points as io
from tutlibs.visualization import JupyterVisualizer as jv
from tutlibs.transformation import Transformation as tr
import inspect

コードは以下のとおりです。

In [8]:
xyz, rgb, _ = io.read('../data/bunny_pc.ply')
vgs_xyz = voxel_grid_sampling(xyz, 0.1)

# visualization
vgs_xyz = tr.translation(vgs_xyz, np.array([1, 0, 0]))
obj_points = jv.point(xyz)
obj_vgs_points = jv.point(vgs_xyz)
jv.display([obj_points, obj_vgs_points])

Output()

上記のコードの出力では、x軸方向に0~1の間にある点群がVoxel grid sampling前の点群、1~2の間にある点群がVoxel grid sampling後の点群を示す。Voxel grid samplingを実装したvoxel_grid_sampling関数では、点群座標値`xyz`とボクセルサイズ`0.1`からVoxel grid samplingを介して、サンプリングされた新たな点群`vgs_xyz`を取得する。

次に、アルゴリズムを確認するため、Voxel_grid_sampling関数の中身を以下に示す。

In [9]:
print(inspect.getsource(voxel_grid_sampling))

def voxel_grid_sampling(coords: np.ndarray, voxel_size: float) -> np.ndarray:
    """Voxel grid sampling

    Args:
        coords: coords (N, C)
        voxel_size: voxel grid size

    Returns:
        samples: sample coords (M, C)
    """
    N, C = coords.shape
    
    # get voxel indices.
    indices_float = coords / voxel_size
    indices = indices_float.astype(np.int32)

    # calculate the average coordinate of the point for each voxel.
    _, voxel_labels = np.unique(indices, axis=0, return_inverse=True)
    df = pd.DataFrame(data=np.concatenate(
        [voxel_labels[:, np.newaxis], coords], axis=1), columns=np.arange(C+1))
    voxel_mean_df = df.groupby(0).mean()

    # use average coordinates as samples.
    samples = voxel_mean_df.to_numpy()

    return samples



上記実装では、返り値がvoxel grid samplingされたサンプルの座標値配列`samples`となる。処理フローは以下の通り。

1. ボクセルサイズに従って、各点をどのボクセルに属するか分ける。所属するボクセルをインデックスとここでは呼ぶ。
2. インデックスごとに点の座標平均値を求める。この平均値がサンプリングの結果となる。